In [ ]:
# mounting google drive to colab
from google.colab import drive
drive.mount('/content/drive')

%load_ext autoreload
%autoreload 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=True)

# Data Loader (Input Pipeline)
batch_size = 64
train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
%cd /content/drive/MyDrive/Deep-Learning-Paper-Practice/Auto-Encoding Variational Bayes

/content/drive/MyDrive/Deep-Learning-Paper-Practice/Auto-Encoding Variational Bayes


In [ ]:
from vae import VAE

vae = VAE(x_dim=784, h_dim1=512, h_dim2=256, z_dim=2)
vae.to(device)

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc31): Linear(in_features=256, out_features=2, bias=True)
  (fc32): Linear(in_features=256, out_features=2, bias=True)
  (fc4): Linear(in_features=2, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [ ]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
from loss import loss_function

In [ ]:
from plot_utils import plot_latent_space

In [ ]:
def train(vae, dataloader, epoch):
    print("="*70)
    print(f"Epoch:{epoch: 3d}")
    print("Train")

    vae.train()

    train_loss = 0
    log_interval = 100

    for batch_idx, (x, _) in enumerate(dataloader):
        optimizer.zero_grad()
        x = x.to(device)
        
        recon, mu, log_var = vae(x)
        loss = loss_function(recon, x, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % log_interval == 0 and batch_idx > 0:
          print("-"*70)
          print(f"| epoch {epoch:3d} | {batch_idx:5d}/{len(dataloader):5d} batch | loss {loss:8.3f}")
    print(f"End of Epoch: {epoch:3d}, Train Loss: {train_loss/len(dataloader.dataset)}")      
    plot_latent_space(vae=vae, n=30, figsize=15, device=device)

In [ ]:
def test(vae, dataloader, epoch):
    print("="*70)
    print("Test")

    vae.eval()

    test_loss= 0
    with torch.no_grad():
        for x, _ in dataloader:
            x = x.to(device)
            recon, mu, log_var = vae(x)
            
            # sum up batch loss
            test_loss += loss_function(recon, x, mu, log_var).item()
        
    test_loss /= len(dataloader.dataset)
    print(f"End of Epoch: {epoch:3d}, Test Loss: {test_loss:8.3f}")

In [ ]:
for epoch in range(1, 11):
    train(vae, train_dataloader, epoch)
    test(vae, test_dataloader, epoch)

Output hidden; open in https://colab.research.google.com to view.